In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Učitavanje podataka o filmovima i ocjenama
movies_data = pd.read_csv('./podaci/movie.csv')
ratings_data = pd.read_csv('./podaci/rating.csv', parse_dates=['timestamp'])

# Spajanje podataka o ocjenama i filmovima
merged_data = pd.merge(ratings_data, movies_data, on='movieId')

# Filtriranje podataka samo za određenog korisnika (npr. korisnik s userId=1)
user_id = 1
user_data = merged_data[merged_data['userId'] == user_id]

# Grupiranje filmova koje je korisnik ocijenio
user_movies = user_data.groupby(['movieId', 'title', 'genres']).mean().reset_index()

# Kreiranje matrice žanrova za svaki film
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_data['genres'])

# Izračunavanje sličnosti između filmova pomoću kosinusne sličnosti
cosine_sim_content = linear_kernel(tfidf_matrix, tfidf_matrix)

# Pronalaženje indeksa filmova koje je korisnik ocijenio
movie_indices = movies_data[movies_data['movieId'].isin(user_movies['movieId'])].index

# Izračunavanje sličnosti između filmova koje je korisnik ocijenio i svih ostalih filmova
similarities = cosine_sim_content[movie_indices].sum(axis=0)

# Sortiranje filmova prema sličnosti u silaznom redoslijedu
recommended_movies_indices = similarities.argsort()[::-1]

# Ispisivanje preporučenih filmova
recommended_movies = movies_data.iloc[recommended_movies_indices]
top_n_movies = recommended_movies[:10]
print(top_n_movies[['title', 'genres']])

                                                   title  \
2532                                   Mummy, The (1999)   
14409    Cirque du Freak: The Vampire's Assistant (2009)   
11487                                         She (1965)   
1189                             Army of Darkness (1993)   
24922              Dragonheart 2: A New Beginning (2000)   
4685                    Megiddo: The Omega Code 2 (2001)   
14374          Aelita: The Queen of Mars (Aelita) (1924)   
9926   Zu: Warriors from the Magic Mountain (Xin shu ...   
23115                 BloodRayne: The Third Reich (2011)   
7328                                  Van Helsing (2004)   

                                                  genres  
2532     Action|Adventure|Comedy|Fantasy|Horror|Thriller  
14409    Action|Adventure|Comedy|Fantasy|Horror|Thriller  
11487  Action|Adventure|Drama|Fantasy|Horror|Romance|...  
1189              Action|Adventure|Comedy|Fantasy|Horror  
24922     Action|Adventure|Comedy|Drama|Fant

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Učitavanje podataka o filmovima i ocjenama
movies_data = pd.read_csv('./podaci/movie.csv')
ratings_data = pd.read_csv('./podaci/rating_4.csv', parse_dates=['timestamp'])

# Spajanje podataka o ocjenama i filmovima
merged_data = pd.merge(ratings_data, movies_data, on='movieId')

# Stvaranje pivot tablice s ocjenama korisnika kao retci i filmovima kao stupci
user_movie_ratings = merged_data.pivot_table(index='userId', columns='title', values='rating')

# Popunjavanje nedostajućih vrijednosti nulama
user_movie_ratings = user_movie_ratings.fillna(0)

# Normalizacija ocjena kako bismo izbjegli efekt različitih skala ocjena
scaler = MinMaxScaler()
normalized_ratings = scaler.fit_transform(user_movie_ratings)

# Izračunavanje sličnosti između filmova pomoću kosinusne sličnosti
item_similarity = cosine_similarity(normalized_ratings.T)

# Pretvaranje rezultata u DataFrame radi lakšeg rukovanja
item_similarity_df = pd.DataFrame(item_similarity, index=user_movie_ratings.columns, columns=user_movie_ratings.columns)

# Odabir korisnika za kojeg želimo generirati preporuke
user_id = 25

# Filtriranje filmova koje je korisnik ocijenio
user_rated_movies = merged_data[merged_data['userId'] == user_id]['title']

# Generiranje preporuka temeljenih na sličnosti filmova koje korisnik već ocijenio
recommendations = pd.DataFrame()

for movie in user_rated_movies:
    similar_movies = item_similarity_df[movie].sort_values(ascending=False).to_frame(name='Similarity')
    similar_movies = similar_movies[similar_movies['Similarity'] > 0]
    recommendations = pd.concat([recommendations, similar_movies], axis=1)

# Grupiranje preporuka prema filmu i izračunavanje prosječnih sličnosti
recommendations = recommendations.groupby(recommendations.index).mean()
#recommendations = recommendations.sort_values(ascending=False)
recommendations = recommendations.T.sum().sort_values(ascending=False).to_frame(name='Score')

# Ispisivanje preporučenih filmova
print(recommendations.head(10))

                                                        Score
title                                                        
Back to the Future (1985)                           50.140155
Forrest Gump (1994)                                 49.488250
Pulp Fiction (1994)                                 49.326906
Raiders of the Lost Ark (Indiana Jones and the ...  49.164845
Groundhog Day (1993)                                49.102916
Matrix, The (1999)                                  48.744419
Men in Black (a.k.a. MIB) (1997)                    48.392760
Silence of the Lambs, The (1991)                    48.274865
Star Wars: Episode V - The Empire Strikes Back ...  47.932182
Jurassic Park (1993)                                47.831171


: 

In [1]:
import numpy as np
import pandas as pd
ratings_data = pd.read_csv('./podaci/rating.csv', parse_dates=['timestamp'])
rand_userIds = np.random.choice(ratings_data['userId'].unique(), 
                                size=int(len(ratings_data['userId'].unique())*0.05), 
                                replace=False)

ratings = ratings_data.loc[ratings_data['userId'].isin(rand_userIds)]
ratings.to_csv('./podaci/rating_4.csv', index=False)

In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

# Učitavanje podataka o filmovima i ocjenama
movies_data = pd.read_csv('./podaci/movie.csv')
ratings_data = pd.read_csv('./podaci/rating_4.csv', parse_dates=['timestamp'])

# Spajanje podataka o ocjenama i filmovima
merged_data = pd.merge(ratings_data, movies_data, on='movieId')

# Stvaranje pivot tablice s ocjenama korisnika kao retci i filmovima kao stupci
user_movie_ratings = merged_data.pivot_table(index='userId', columns='title', values='rating')

# Popunjavanje nedostajućih vrijednosti nulama
user_movie_ratings = user_movie_ratings.fillna(0)

# Normalizacija ocjena kako bismo izbjegli efekt različitih skala ocjena
scaler = MinMaxScaler()
normalized_ratings = scaler.fit_transform(user_movie_ratings)

# Izračunavanje sličnosti između korisnika pomoću kosinusne sličnosti
user_similarity = cosine_similarity(normalized_ratings)

# Pretvaranje rezultata u DataFrame radi lakšeg rukovanja
user_similarity_df = pd.DataFrame(user_similarity, index=user_movie_ratings.index, columns=user_movie_ratings.index)

# Odabir korisnika za kojeg želimo generirati preporuke
user_id = 25

# Pronalaženje sličnih korisnika
similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]  # Izbjegavanje samog korisnika

# Filtriranje filmova koje su ti korisnici ocijenili, a koje korisnik još nije gledao
movies_watched_by_similar_users = user_movie_ratings.loc[similar_users.index]

unwatched_movies = movies_watched_by_similar_users.loc[:, ~user_movie_ratings.loc[user_id].astype(bool).values]

# Izračunavanje prosječnih ocjena za svaki film
average_ratings = unwatched_movies.mean()

# Sortiranje filmova prema prosječnim ocjenama u silaznom redoslijedu
recommended_movies = average_ratings.sort_values(ascending=False)

# Ispisivanje preporučenih filmova
print(recommended_movies.head(10))

title
Star Wars: Episode IV - A New Hope (1977)                1.592012
Fugitive, The (1993)                                     1.395493
Toy Story (1995)                                         1.374837
Apollo 13 (1995)                                         1.337859
Star Wars: Episode VI - Return of the Jedi (1983)        1.326015
Star Wars: Episode V - The Empire Strikes Back (1980)    1.313953
Fargo (1996)                                             1.266647
Sixth Sense, The (1999)                                  1.138524
Independence Day (a.k.a. ID4) (1996)                     1.111801
True Lies (1994)                                         1.060884
dtype: float64
